## BASH CLI Cheat Sheet

**Vertical bar**: The vertical bar | represents a pipe in Linux/UNIX systems. It is a form of redirection that sends the output of one program to the input of another program. So for example, `cat file1 | wc` means that the command `cat` will read the contents of `file1` and the send the output as input to the `wc` command. So anytime there is a vertical bar means that there is are multiple commands forming a chain, almost like currying in programming.

**Single dot**: A single dot, aka a period, in a path refers to the current directory. Double dots refer to the parent directory.

**cat**: The `cat` command refers to concatenation and is used for viewing text files in the terminal itself. Example usages below:

 `cat file1 file2`: Shows contents of file1 and file2 in order. Multiple files can viewed this way.
 `cat -n file1`: Shows contents of file1 but also labels each line number so you know which line number a certain piece of text appeared.
 `cat > newfile`: Creates a new file named 'newfile'. When doing this, the shell will expect you to enter some initial text to fill the file. You can simply exit early to not enter anything.
 `cat file1 > file2`: Copies file1 to file2. file2 is overwritten if it exists, or created if it doesn't.
 `cat file1 file2 file3 > file4`: Copies file1, file2, file3 to file4 in order or creates a new file called file4 with the copied contents.  
 `cat file1 >> file2`: Appends file1 to file2.
 `cat >> file1`: Will append anything typed in the terminal to file1.
 `cat < file2`: Provides file2 as an input to the `cat` command which will just read out file2.
 `cat file1 | more`: Shows as much content from file1 in the terminal as possible and if it doesn't fit, will ask user to if they want to see more.

**source**: The `source` command in BASH means to run a file in the current SHELL instance instead of starting a new shell. So `source filename` will run the file in the current shell. A short hand for this command is `. filename`. Notice the space between the period and the filename.

**touch**: The `touch` command is used to create files and change the timestamps/access times of files. So `touch filename1 filename2` creates 2 **empty** files. `touch -c filname` is used to check if a file is created; if it is not, don't create it, just check. 

**./**: This command is similar to the `source` command but will run the file in a new shell instance. So `./filename` runs the file in a new shell.

**> or <**: This isn't so much a command but a redirection operator. It controls the input and output of commands. So if I have a command called `foo` then `foo < filename` will provide the `filename` as the input to the `foo` command. Similarly, `foo > filename.txt` will redirect the output to be put into a text file called `filename.txt`. Using `foo >> filename.txt` will append the output of `foo` to `filename.txt`.

**>>**: As stated above, this redirects the output of a command to a file and appends the output to the file. 

# GIT AND GITHUB NOTES

## BASIC GIT TERMINOLOGY AND COMMANDS

**git command structure**: git commands follow the general structure of `git command --options path`. Command refers to the name of the command and options are the optional actions a command can perform. Options always come after the command and have either `-` or `--` before the option name; you can have multiple options for a single command. Path is either the path to a directory or a file that the command should act on. Git documentation usually specifies option in brackets `[]` and directories in angled brackets `<>`. So a command such as `git rm --cached myfile.txt` will be presented in documentation as `git rm [--cached] <directory>`. Some options require a directory as an argument and the documentation will list it as `[--option=<directory>]`.

**Repository**: The basic container for the project, usually in the form of a directory. Git will track any changes to the directory. It will not track changes made to the parent directory. When creating a repository it is automatically set as the master branch. To set a current directory as a repository (aka create a repository), use `git init`.

**Origin**: A commonly accepted term used to indicate the "original" repository, aka the very first repository. You can refer to a local origin or a remote origin. Local origin is the original repository on the local machine. Remote origin is the repository that you cloned from.

**Commits**: Basically a save-point of the files inside a repository. When making a commit, you save the current status of all the files. Making multiple commits builds up a commit history like a sequence of events. There are three stages to making a commit:

1. Save the file normally without involving git.
2. Staging: When a file is going to be committed, it needs to be added to a staging area. This is like a buffer area before we actually make the commit and let's us look over things before we commit. Here are some commands regarding the staging area:
  * `git status` = get info about what files are currently staged
  * `git add [filename]` = add the file to the staging area
  * `git add .` = add all files in the repository that have been modified to the staging area
  * `git rm --cached [filename]` = remove the file from the staging area
3. Commit: Make a commit and save the current status of the files. Here are some commands for making a commit:
  * `git commit -m "[message]"` = commit the files in the staging area. the option `-m` means to add a message to describe the commit. It is standard to always add a message. Be specific in the message about the commit. When this command is executed, it will print to console the number of lines deleted. This doesn't mean any code was actually deleted. It just detects changes as though you deleted the code and then added it back.
  * `git log` = display the full commit history of the repository.


## GIT OBJECTS

Git is just a giant key-value data store modeled as a linked-list. When something is inserted into a git repo, git returns a unique key that is used to retrieve the stored item. These keys are also used as the pointers within git's linked list structure. Git stores these items as "objects". There are different types of objects:

  * commit = store information about the commit
  * tree = store information about a directory
  * blob = file content stored as raw binary data; blob is short for 'binary large object'
  * annotated tag = store tag information

Each object has a unique hash to identify it, made using SHA-1 (secure hash algorithm). The hash is generated using the contents of the object. So for a source code file, it literally uses the source code itself to generate a hash. This pretty much guarantees every object has a unique hash. For tree objects, which refer to directories, it uses the structure of the directory to calculate the hash.

Now git organizes the object types in the following way:

```
commit
  |-- tree
        |-- blob
        |-- blob
        |-- tree
              |-- blob

```

The commit object has a hash that identifies a tree object. That tree object has 3 hashes: 2 for blobs, 1 for another tree. That second tree object contains a hash for another blob object. Therefore, git simply stores everything in a giant graph structure. You can view the objects it currently stores in the .git/objects directory. The objects folder will list a bunch of folders, with each folder name only being 2 characters long. Specifically, when git generates a hash for an object, the hash will be 40 characters long. It will take the first 2 characters and use that as the name of the folder and store the remaining 38 characters as a file in that folder.

Let's talk more about how git generates hashes for objects. For blob objects, it prefixes the object content with the word "blob" followed by the object content's length in regular numerical numbers followed by a null character.

Tree hashes are different. Trees objects contain one line per file or subdirectory/subtree with each line having the following format:
`file-permission object-type object-hash file-name`
Example is: `100644 blob e69de29bb2d1d6434b8b29ae775ad8c2e48c5391 index.txt`

File permission here is 644 (ignoring the 100), and permissions of 644 mean that owner has read/write access of the file while others can only read. Permissions of 600 mean owner can read/write but other people can't do anything. To generate the tree hash, it uses all this information to feed into the SHA-1 algorithm.

As for commits, their hashes are generated using the information contained in them:
  * parent commit hash, so each new commit points to the previous one. May or may not be present depending on commit history.
  * starting tree hash
  * author
  * committer
  * date (including time)
  * commit message

## GIT BRANCHES AND REMOTE BRANCHES

### MASTER BRANCH IS NOW CALLED MAIN BRANCH ON GITHUB

A branch is simply a movable pointer to a commit within a repo; basically the hash of the commit is used as the pointer. Each time a Branch info is stored in .git/refs/heads directory. There are also files located within /.git called "HEAD", and "ORIG_HEAD", and "FETCH_HEAD", and possibly other "HEAD" files.

  * "HEAD" is a file that a pointer to the current branch you are on. Doesn't actually use a hash as a pointer but rather references the file located in .git/refs/heads that contains the pointer. So "HEAD" is basically a pointer to a pointer; it points to the current branch which is also a pointer.
  * "ORIG_HEAD" is the previous commit that "HEAD" was pointing to.
  * "FETCH_HEAD" is the pointer to the tip of the remote branch. So it locally tracks where the remote branch has advanced.

To create a new branch, use the command `git branch <branch-name>`. This creates a new file in the .git/refs/heads directory whose filename is the name of the branch and the contents of the file is the hash of the commit that the previous branch was pointing to when the new branch was created. However this command won't move you to the new branch after the branch is created. To move onto the new branch use: `git checkout <branch-name>`. Now the "HEAD" file will point to the new branch we created.

Now what happens if we make a commit after we made a new branch and switched to it? Well, the new branch will move its pointer to point to the new commit but the previous branch will not move to point to the new commit. Diagrammatically it can look like:

```
//before new commit
                      master
                        |
commit1 <- commit2 <- commit3
                        |
                      new branch
                          |
                        HEAD

//after new commit
                      master
                        |
commit1 <- commit2 <- commit3 <- commit4
                                    |
                                  new branch
                                      |
                                    HEAD

```

So the previous branch, master in this case, will remain stationary but the new branch will move. If we move back the master branch, then the "HEAD" file will point to the master branch which in turn points back to a previous commit.

We can also make a new commit while working back in the old branch. Any new commits made there will have the old branch move to the new commit as expected. So in the above example, we can make a new commit in the master branch. This will add a new commit that will point to commit3 in addition to having commit4 point to it. This means the commit history will look like:

```
//after new commit on master branch

                            HEAD
                              |
                            master
                              |
                          new commit5
                          /
commit1 <- commit2 <- commit3 <- commit4
                                    |
                                  new branch

```

Now what happens if we want to reset back to an old commit? We simply move the branch pointer back to the previous commit. We can do this with the command `git reset <commit HASH>`. Instead of using the hash to explicitly specify the commit we move to, we can do a relative move: `git reset current~<num>`. So `git reset current~2` means to move back 2 commits relative to the current commit we are currently on. So in the above example, if we were on commit4 in the new branch, then `git reset current~2` moves us back to commit2. Just remember that `~` means to move back and `^` means to move forward. So `git reset current^2` moves forward 2 commits.

What if we just want to view a previous commit, but not reset the code back? We use `git checkout <branch>` or `git checkout <commit>`. This simply moves the "HEAD" pointer around to the specified branch or commit.

As for remote branches, the branches in the remote repo, git needs to do a bit of coordination between those branches and the current branches on your local computer.

## GIT REFS

A ref is just a file containing a commit hash. So it is a shorthanded way of referring to a specific commit. This is used so that each branch pointer knows which commit to point to. Located in .git/refs, we can see various ref files. As discussed in previous section, the refs/heads directory contains a bunch of files detailing the different branches in the repo; the filename is the name of the branch. But those aren't the only refs. There are different kinds of refs:

  * packed refs: a compression of a bunch of refs into a single file. It is basically a line by line file with each line containing the directory location of the ref and the hash the ref contained.
  * special refs: like the "HEAD", and "FETCH_HEAD" files. The "HEAD" file points to the ref file that contains the commit hash. So it points to a pointer. 

When specifying a ref you can either use the short name, like the name of the branch, or the full name which is basically the full directory path such as refs/heads/master.

A refspec is a mapping of a source branch to a target branch. It has syntax of `<src>:<target>`. So if one wanted to `git push` the local master branch to the remote master branch, we could use: `git push origin master:master`.

## CONNECTING TO GITHUB: PUSH/PULL/FETCH/CLONE/FORK AND OTHER THINGS ABOUT WORKING WITH THE REMOTE REPOSITORY

Github hosts a copy of the repository on the cloud, referred to as the remote repo. To get files and put files into the remote repository, there are several commands that will be useful:

1. `git clone <git-url> <dir>`: This command allows you to copy everything from the remote repository to your computer. This will create a new folder in `<dir>` whose name will be the same as the name of the repository you are copying. You cannot clone into an existing directory if it is not empty. Here are some useful options with this command:
  *

2. `git push <repo> [<refspec>]`: Command to update the remote refs using local refs and sending over any necessary objects to complete the change.


## GITIGNORE

There are some files/directories in your project that you may not want git to commit. This includes compiled code, dependency caches, files generated at runtime, etc. To stop git from working with these files/directories, you make a .gitignore file. The syntax of .gitignore files follows that of Linux globbing patterns. Use `touch .gitignore` to make the file as Windows sometimes has issues with files that only have an extension in their name. 

Example ignores:

`bar/`: Ignores the `bar` directory relative to the .gitignore file. So if the .gitignore file is in the `foo` directory so `foo/.gitignore` and so is `bar`, so `foo/bar`. Then `bar/` will ignore the `bar` directory that is located in the same directory as .gitignore. 

`/bar/goo`: Ignore the `goo` directory that is within the `bar` directory with the `bar` directory being at the same level as the .gitignore file. 


## GIT COMMAND SHEET

**`git remote -v`**: View the remote origin associated with the current repository.

**`git branch`**: View all branches on the local repository. 

**`git show HEAD`**: Shows the commit that the HEAD pointer is currently pointing to.

**`git show-ref`**: Show the refs available on the local repository along with the commit IDs. By default it shows the tags, heads, and remote refs. 

# FILENAME/DIRECTORY STRUCTURE INFORMATION

**~**: Represents the home directory. For Windows systems, this usually defaults to `c:/users/username`.

**period in front of directory**: Indicates the directory is "hidden" from normal view. Also known as dot files or dot directories, this convention comes from Unix and Linux systems as a a way to hide configuration files from the user.

**symbolic link/symlink/soft link**: A file that contains a reference to another file or directory in the form of an absolute or relative path. 

**canonical path**: This is the ''true'' path of the file or directory starting from the root directory. There is only one canonical path for every file or directory.

**absolute path**: This is the path starting from the root directory but it allows for path manipulations like up-level references such as `c:/foo/bar/../example.txt`. Here `example.txt` is located in the `foo` directory since the `..` sends it back to the parent directory. The canonical path is just the shortest absolute path. So for the above example, it would read `c:/foo/example.txt`, in which it removes the additional path manipulations. So there can be multiple absolute paths but only one canonical path. 

**ntpath**: For windows systems, the path components are separated by a **double** backslash

**posix path**: Unix systems have 

## PYTHON OS.PATH MODULE

The path structure is dependent on the operating system that Python is running on. So Windows machines will naturallly use `ntpath` and Unix uses `posixpath`. If you want to use one specific convention over the other, you can instead import `posixpath` or `ntpath` which will provide the same interface as the `os.path` moodule. So you can use `import posixpath` to use the posix style path convention on a Windows machine if you want. 

`os.path.realpath`: Derefernces symbolic links to get the canonical path

`os.path.split`: Splits path into tuple of (head, tail). The tail part will never contain a slash so if the path ends with a slash, then the tail will be empty. 

`os.path.normpath`: Takes a pathname and collapses redundant separators and up-level references. So on a posix system, `a//b` and `a/b/` and `a/./b` and `a/foo/../b` become `a/b` since the extra slashes or up-level references are redundant. On Windows, it can be used to convert forward slashes in path names to backward slashes. 

# JS NOTES AND INFORMATION FOUND HERE

## JS PROTOTYPE INHERITANCE

Information here is based on ECMAScript 5, but many concepts should still apply in future ECMAScript versions.

In the context of JS, all objects are ECMAScript objects. In other words, JS objects are implementations of what an "object" should be based ECMAScript standards. Therefore, the browser/JS environment has a bunch of behind the scenes data structures/functions that it uses to emulate the JS object. We shall refer to this background 'stuff' as an ECMAScript object. This means there are a lot of internal/private functions and properties associated with JS objects that we do not have access to when programming because they are kept behind the scenes. These private properties/functions, as stated before, follow the ECMAScript guidelines.

Here are some of the internal methods and properties associated with ECMAScript objects:

1. [[Class]]: Specifies what 'type' of ECMAScript object
2. [[Prototype]]: Specifies what ECMAScript object is the prototype object
3. [[Call]]: Internal method that runs the source code associated with the ECMAScript object
4. [[Code]]: Source code associated with the ECMAScript object
5. [[Construct]]: Internal method that creates a new ECMAScript object and runs the internal [[Call]] method as well.

ECMAScript objects are broadly split into two classes: [[Class]] = 'Object' or [[Class]] = 'Function'. ECMAScript objects with [[Class]] = 'Function' will have its internal [[Code]] property set equal to the source code of the function. So if you write a function declaration in JS, you are simply creating an ECMAScript object with [[Class]] = 'Function'.

Now in every JS run-time environment there are two global ECMAScript objects whose names are literally "Function" and "Object". There is and will only ever be one copy of these objects. Obviously the object called "Function" has [[Class]] = 'Function' and the same logic goes for "Object".

Now in ECMAScript every object has another ECMAScript object, `null`, or primitive data as its prototype, aka its parent object. Therefore in addition to "Object" there is another separate ECMAScript object whose name is literally "Object.prototype". Similarly, the "Function" object has the ECMAScript object "Function.prototype" associated with it.

Now in order to associate one ECMAScript object as the prototype for another ECMAScript object, we need to set the internal [[Prototype]] property of the second object to point to the first object. What is interesting here is that the "Function.prototype" object has its [[Prototype]] object set to point to "Object.prototype". Meanwhile, "Object.prototype" has its [[Prototype]] -> `null`. So far the prototype chain looks like:

```
      null
       |
"Object.prototype"
       |
"Function.prototype"
```


Quick side note: "Function.prototype" is a function that accepts any value and simply returns `null`.

Now as for "Object" and "Function", they have their [[Prototype]] set to point to "Function.prototype". However, ECMAScript objects also have an external/public property called 'prototype'. This is the property that we can programmatically access and is different from the internal [[Prototype]]. So "Object".prototype is a property of "Object" while "Object.prototype" is an entirely separated ECMAScript object. Now "Object".prototype is set to point to "Object.prototype" and similarly "Function".prototype is set to point to "Function.prototype". So the full prototype chain thus far looks like:

```
                                          null
                                            | I
                                    "Object.prototype"
                                            |         \
                                            | I        \ E
                                            |           \
          "Function".prototype --> "Function.prototype"  "Object".prototype
                              \            /\             /
                             E \        I /  \ I         / E
                                \        /    \         /
                                "Function"      "Object"

  E = external property, I = internal property, aka [[Prototype]]

```

Whenever JS does anything which involves backtracking up the prototype chain, it uses the internal [[Prototype]] property to look up the prototype objects, not the public `.prototype` property.

This is why when we use `instanceof` we find "Object" is `instanceof` "Function" and "Function" is `instanceof` "Object". What `instanceof` does is it looks at the second argument's public prototype property, finds the associated object, and then checks if the found object is located in the prototype chain of the first argument. So `Object instanceof Function` will access "Function".prototype, see that it points to "Function.prototype", and then check to see if "Function.prototype" appears in the prototype chain of "Object", which it does.

## JS FUNCTION DECLARATION  

When we declare a function, such as `function myfunc() {}`, we are actually telling the JS runtime to create a new ECMAScript object with its internal [[Class]] = 'Function'. We also set [[Prototype]] = "Function.prototype". In addition, this new ECMAScript object is created with public properties called 'prototype' and 'constructor'. The public 'prototype' property is set to point to another newly created ECMAScript object whose internal [[Class]] = 'Object'. This other object is referred to as our function's prototype object. So using the above example, in addition to getting a new ECMAScript object called "myfunc", we would also get another ECMAScript object called "myfunc.prototype".

As for the 'constructor' property of "myfunc", this is set to point to whatever function is called to create the ECMAScript which in this case will be "Function". The reason for this is because we are declaring a function and "Function" is used to create new ECMAScript function objects. In addition, "myfunc.prototype" also has a public 'constructor' property which points to "myfunc". So we would call "myfunc" to create "myfunc.prototype".

Diagrammatically the situation looks like:

```
  ------------------------------------------           ------------------------------------------
  |  * name: "myfunc"                       |          | * name: "myfunc.prototype"             |
  |  * [[Class]] = 'Function'               |          | * [[Class]] = 'Object'                 |
  |  * [[Prototype]] = "Function.prototype" |          | * [[Prototype]] = "Object.prototype"   |
  |  * .prototype = "myfunc.prototype"      |          |                                        |
  |  * .constructor = "Function"            |          | * .constructor = "myfunc"              |
  -------------------------------------------          ------------------------------------------

```

Here we see that there is a bit of a circular connection between the two ECMAScript objects. "myfunc".prototype points to "myfunc.prototype" and "myfunc.prototype".constructor points back to "myfunc".

"Function.prototype" and "Object.prototype" both also have a public property called constructor which point back to "Function" and "Object" respectively.

## JS new KEYWORD

Let's say we declared a function `function myfunc(){}`. Next we use `new myfunc()`. The `new` keyword instructs the runtime to create a new ECMAScript object. This new ECMAScript object will be an instance of "myfunc". The `new` keyword basically calls the target ECMAScript object's internal [[Construct]] method; the target in this case is "myfunc". The [[Construct]] method is set to behave roughly as follows:

1. Create a new ECMAScript object
2. Set the new ECMAScript object [[Class]] = 'Object'
3. Get the public prototype property of the object that is doing the creating. In this case, the object doing the creating is "myfunc", so it would access "myfunc".prototype.
4. If the gotten property is an object data type, then set the internal [[Prototype]] of the new ECMAScript object to point to it. So in this case the internal [[Prototype]] of the instance is set to point to the object that "myfunc".prototype points to, which is "myfunc.prototype".
5. If the gotten property is not an object data type, set the internal [[Prototype]] to "Object.prototype". So if the public property somehow pointed to a primitive like a boolean, correct the error and force [[Prototype]] to point to [[Object.prototype]]
6. Invoke the [[Call]] method of the constructor ECMAScript object. In this case it would call the [[Call]] method of "myfunc" since "myfunc" is the object doing the creating.
7. If the [[Call]] method returns a value that is an ECMAScript object, use this value as the output of the [[Construct]] method. So if somehow the source code of "myfunc" returns an object, then this object is used as the output of [[Construct]] instead of the object that was created in step 1. This doesn't mean the object created in step 1 goes away however; it is still present until it is garbage collected.
8. If the [[Call]] method doesn't return anything, then return the original ECMAScript object created in step 1.

Let's say we have the following code: `var foo = new function(){//code goes here}`.
What happens here is that first the runtime environment will recognize that there is a function declaration in this assignment so it will go through the steps of creating a new ECMAScript object with [[Class]] = 'Function'. Here the ECMAScript object won't have a name since we didn't provide one in the declaration; it will have some internal labelling however that we won't ever know about. Next the `new` keyword takes effect and invokes the [[Construct]] method of the unnamed ECMAScript function object. This creates a new ECMAScript object. If the code of the unnamed function somehow returns an object, say an object literal `{}`, then this object will be assigned to `var foo` instead of the originally created ECMAScript object. If it doesn't return anything, then the original ECMAScript object is returned and assigned to `foo`.


## ECMASCRIPT DATA TYPES

There are different data types defined by the ECMAScript standards. This can be broken into two broad categories: language and specification.

Language data types refer to things that the programmer can use in his program. So this includes types like boolean, null, number and the ECMAScript objects we have talked about. Any type of data that the programmer can manipulate falls in the category of the langauge data types.

Specification data types are a classification used behind the scenes by the runtime environment that a programmer does not have access to. It is sort of like meta-data and is used in determining how the source code should be executed and its current state.


## JS EXECUTION CONTEXT/SCOPE CHAIN/CLOSURES

To start this discussion, we will need to know about a certain specification type data structure: the lexical environment. This data structure is based on 2 things:

1. A reference to an outer lexical environment, aka an outer scope. So one lexical environment can encapsulate another.
2. An environment record: Some behind the scenes data structure that is used to keep track of the list of bindings. Bindings refer to the association between a data value and the name used to identify the data value. The environment record tracks identifier names, the values and the association between them. There are 2 types of environment records:
  * Object: Record the bindings associated with objects such as object properties
  * Declarative: Record the bindings associated with function and variable declaration and catch clauses.

So a lexical environment is basically a set of bindings, grouped and ordered in a way that matches the nesting structure (scoping) of the source code.

Now JS code is carried out within an execution context. The exec. context is an abstract idea. It is the process by which source code should be evaluated. So the source code should be executed in a specific way and have certain things associated with the execution process.

For ECMAScript it says that an exec. context should have 3 things associated with it:

1. Lexical environment: Basically some sort of scope. This is needed so that the runtime environment knows how to resolve identifier names and associated values and know which variables are part of the current scope. A loop shares or is part of one lexical environment. So each iteration through a loop doesn't create a new lexical environment.
2. Variable environment: A reference to a lexical environment whose environment record contains the bindings created by variable/function declarations while processing source code.
3. `this`: The current ECMAScript object that serves as the value of `this` within the current exec. context.

When an exec. context starts the lexical environment and variable environment are the same, but the lexical environment can change while the variable environment stays the same. One more thing to mention is that all ECMAScript objects with [[Class]] = 'Function' have an internal property called [[Scope]]. This property points to the lexical environment associated with the function and is set when the function ECMAScript object is created. Specifically, the value is set to the variable environment of the current exec. context in which the function was declared.

Execution contexts form a stack structure in which while one execution context is happening, it can push another execution context onto the stack and that new one takes over. Each function call creates a new execution context. This is important for closures as it means each time we call the function can create a new closure.

Let's say we call a function in the global scope. The execution context will be set up approximately as follows:

1. Create new exec. context to evaluate the function.
2. Set the `this` value. If the function is a property of an ECMAScript object with [[Class]] = 'Object', then set the `this` to be equal to that object. So if the function is a method of an object, `this` refers that object. Otherwise set it to another ECMAScript object, usually the global object. So any plain function which does not have an explicit `this` value will have `this` refer to the global object.
3. Call an abstract internal method known as NewDeclarativeEnvironment with [[Scope]] of the function as the argument. This creates an empty lexical environment with the reference to the outer lexical environment set to [[Scope]]. Recall that when we declared the function that [[Scope]] was set to the variable environment of the exec. context in which the function was declared. So here we called the function in the global scope so that means the empty lexical environment we created has its reference pointing the global execution context since the function was declared in the global scope.
4. Set the lexical environment and variable environment of the function's exec. context to the newly created lexical environment in step 3. Remember that an exec context has two properties: lexical and variable environments. We set both properties to point to the newly created lexical environment.
5. Create bindings for the functions/variables declared inside the current function's exec context which are added to the exec. context's variable environment's environment record. This is the hoisting part. We create the bindings for the declarations before actually running the source code.

Let's see how this works out in an example.

```js

function func1(){
  function func2(){
    return 2;
  }
}

func1();

```

Here we declare `func1` in the global execution context so we create a new ECMAScript object with [[Class]] = 'Function' and [[Scope]] = variable environment of global execution context. Next we actually run `func1` which creates a new execution context. Since no `this` value is given, `this` defaults to the global object. Now create a new lexical environment and set the lexical environment and variable environment properties of the current execution context to this newly created environment. This newly created lexical environment will have as its outer reference whatever [[Scope]] points to which in this case is the global execution context variable environment. Next we create a binding for the declared `func2`. To bind `func2`, create a new ECMAScript object with [[Class]] = 'Function' and set the binding within the environment record of the variable environment of the current execution context. Diagrammatically it looks like the following stack:

```

-----------------------------------------------------------------
lex, var env. properties = new lex. env -> which has outer lex that
points to the [[Scope]] of func1.
                                                                            func1 execution context.
var env. has binding for: func2 -> [[Scope]] of func2 points to             
this exec. context's var. env. property.
-----------------------------------------------------------------
lex, var env. properties = global env.

global var env. has binding for: func 1 -> [[Scope]] of func1                global execution context.
points to this exec. context's var. env. property.
------------------------------------------------------------------

```

This explains why closures work. For example, take the following code:

```js

function func1(){
  function func2(){
    //code here
  }
  return func2;
}

var f = func1();
```

Here the [[Scope]] of func2 is set to be the newly created lexical environment of func1 when func1 is called. Therefore when func2 is assigned to `var f`, the lexical environment that was created during calling func1 must persist because func2 still references it with its internal [[Scope]] property. This means that the lexical environment can't be garbage collected since it still has a reference pointing to it. This also means that anything that the lexical environment contains, such as bindings to other function/variable declarations, also persist. Just remember that each time a function is called a new execution context is created and a new lexical environment is created to go with that execution context.


## JS CLASSES

Inside a class declaration, we can define methods. There are two types of methods: static and prototype. Take a look at the following class definition:

```js

class Rect {
  constructor(h, w){
    this.h = h;
    this.w = w;
  }

  static name(){
    return 'rectangle';
  }

  area(){
    return this.h * this.w;
  }

}

```

In here, the static method `name` refers to a function that is bound to the function object `Rect` itself. In other words, it is a traditional class function that must be called as `Rect.name()` and not through an instance of `Rect`. So if we had an instance `let r = new Rect(1,1);` and tried `r.name()`, it would fail.

The function `area` is a prototype method. This function is set as a method of `Rect.prototype`. That means when we call `r.area()`, it will not find a function
`area` inside the instance `r` but will have to go up the prototype chain to `Rect.prototype`. The execution context will be passed along, in this case referring to the object instance `r`, and things will proceed as normal.

However, this isn't necessarily a good thing all the time. When calling the method through an instance, the `this` value is set to the calling instance. However, if we were to say reassign the function say like `let a = r.area` then we lose the execution context or the `this` value and we can no longer call `a()`. The same thing is true if we try to assign `let n = Rect.name`. This is because all syntax inside the class declaration operates in strict mode which means that `this` no longer defaults to the global window object. Contrast this to the traditional method of defining constructors in which `this` will default to the global object. This is a problem because sometimes we need to use these prototype methods as callbacks and when we provide the function, it loses its context or `this` value. This happens because when we provide a function as an argument, the js engine will set the function to the parameter name. So if we had: `function(callback)` and we provided `r.area`, then the assignment `callback = r.area` will happen and we lose our context.

We can solve this manually by creating a property in the constructor that refers to a function with an explicitly set `this` value. For instance, we can use:

```js

class Rect {
  constructor(h, w){
    this.h = h;
    this.w = w;
    this.area = this.area.bind(this);
  }

  static name(){
    return 'rectangle';
  }

  area(){
    return this.h * this.w;
  }

}

```

Here what happens is we create a new property in the instance called `area` and in order to set it, we go up the prototype chain to get the `area` property from `Rect.prototype`. Now the instance property `area` is set to perform the same code as the prototype method `Rect.area` but the execution context is fixed to always be the calling object. So now if we have `let a = r.area;`, then when we call `a()`, it will return the correct area of the original instance object because the execution context is fixed to always point to that object. This is useful for callbacks since now we can provide the function as a callback and when it executes, the `this` value will still be pointing to the original object. In addition when we call `r.area`, we no longer go up the prototype chain anymore since the instance object already has the property `area`.


Another thing about js classes is the creation of variables inside the constructor but are not bound to the instance. For example:

```js

class thing{
  constructor(v){
    const val = v;
  }
}

```

Here if I call `let t = new thing('foo')`, then the constructor will execute and a new variable `val` will be created with its value set to `v`. This object will continue to persist but we will have no way of programmatically accessing it because it is confined to the scope of the constructor. So it lives on in the background. One question is what happens when we create multiple instances? Then the constructor is called multiple times. While one might think that since `val` has the keyword `const` that it can't change value, what really happens is that different versions of `val` are created, each of which is `const`, but are differentiated by their different execution contexts.


## JS PROMISES

A promise is a proxy for a value that is not known (aka undefined) at the time the promise is created. Therefore asynchronous functions can return a promise in place of the actual value. This makes the function seem like it is synchronous. Promises can be pending, fulfilled, or rejected. If fulfilled or rejected, the promise will execute a callback function that was supplied to the Promise's `then` method. If a promise is fulfilled or rejected before a callback is supplied to the `then` method, it is okay. The callback will be executed once the callback is supplied so there is no need to worry about a race condition in which you need to make sure the callback is supplied before the promise has been fulfilled or rejected.

The promise in JS is different from lazy evaluation. Promises represent processes that are already happening, not delaying a process until later. Basically, a promise is used for things already going on, and we are waiting for the result. Lazy evaluation is waiting until later to get things started.

Promises use the Job Queue, which is like the message queue in the event loop except that items in the job queue take priority over the message queue. So if there is anything in the job queue, it will execute first before anything in the message queue.


## JS LAZY EVALUATION

Sometimes we don't want the result of a function until it is needed. However, we sometimes need to reference the result beforehand. This creates a bit of a problem since we want to reference the result but we can't get the result unless we call the function but we don't want to call the function just yet. We can deal with this problem via lazy evaluation. Basically, we won't call the function until we need to actually use the result for something. In addition, we can use memoization to remember the result after it has been evaluated so we don't have to evaluate the function again. This is useful if we only want one result from the function. We can implement a lazy evaluation in JS with closures.

```js
  const lazy = function( func ){
    let res = undefined;
    let finished = false;
    return function(){
      if(finished) {return res;}
      res = func.apply(this, arguments);
      finished = true;
      return res;
    }
  }

  let result = 0;
  let lazyResult = lazy(() => {
    result += 10;
    return result;
  });

  console.log(lazyResult());
  console.log(result);
  console.log(lazyResult());

 ```

If we evaluate the above code, then we would get the console printing out 10, three times. The lazy function is basically a decorator that uses a closure to remember the result of the function we want to evaluate. It returns a function that will call our original function but only if we haven't called the function yet. If we have, then it simply returns the stored result back. This is why even though the original function (the arrow function) supposedly increments the result by 10 each time, the output of the lazyResult will still be 10. The result however, because it is in a larger scope, will still increase but lazyResult won't.


## SHALLOW COPY vs DEEP COPY

A shallow copy of an object means that if the object contains any references to other objects, only the references are copied, not that actual objects they refer to. A deep copy means that both the references and the objects that the references point to will be copied.

Take the following example objects:

```js

let obj1 = {};

let obj2 = {obj: obj1};

```

If we made a shallow copy of `obj2`, then we get a new object whose `obj` property will point back to the original `obj1`. That means any edits to `obj1` will affect `obj2` and the copy of `obj2`. If we made a deep copy of `obj2`, then we get a new object whose `obj` property points to a copy of `obj1`. So now there are two versions of `obj1`. This means that editing the original `obj1` will only affect the original `obj2`; the copies of these objects will not be affected.



## JS EVENT LOOP
Recall that function calls are evaluated on a stack structure. When a function is going to be evaluated, it is pushed onto the stack. When the function is finished/returns a value, it is popped off the stack. Therefore, the event loop at each iteration will check if the call stack has something, evaluate the topmost function in the stack, then when it is finished, pop the function off the stack.

Now this is simple enough to understand, but things get weird when we start using asynchronous functions like setTimeOut.
For example, look at the following code:

```js

  const bar = () => console.log('bar');

  const baz = () => console.log('baz');

  const foo = () => {
  console.log('foo')
  setTimeout(bar, 0)
  baz()
  };

  foo();
```

Here we will see that the console will print out 3 messages in the following order:
1. foo
2. baz
3. bar

This will happen even though we set the timeout to be 0 seconds which implies that it should have executed immediately. The reason for this is due to the message queue. The message queue is a data structure used by JS runtimes (web browser or node.js) to handle asynchronous events like user-initiated events or fetch responses or DOM events. The event loop however gives priority to the call stack over the message queue. So what happens with the previous code is as follows:

1. Push foo onto stack
2. Push console.log onto stack
3. Evaluate console.log and pop off
4. Push `setTimeOut` onto stack
5. Evaluate `setTimeOut`. This puts a message into the message queue with bar as the callback function. Pop off `setTimeOut`.
6. Push baz onto stack, evaluate, and then pop off.
7. Pop off foo since it is finished.
8. Call stack is empty, check message queue.
9. Process message queue, which involves pushing bar onto the call stack.
10. Evaluate bar, and then pop off.

A key thing to note is that the function `setTimeOut` makes a call to the browser API. In other words, it asks the browser to do the waiting while the call stack and message queue proceed forward normally. So the waiting happens in another thread/process which is handled by the browser. The browser is inherently multi-threaded.

Here's another example. Let's say we ask for data from a server using `fetch()`. Let's also say we have a button that executes a callback that is supposed to work with the data that we get from the server. Let's say that our connection speed is slow so that `fetch()` takes a while. Let's say our pseudo-code looks like

```
 * bind callback to button
 * immediately after binding, execute fetch to get data
```

Is it possible for us to execute the callback while the fetch gets the data? The answer is no. What happens is the following: The browser will notice that the user has clicked a button and will load the callback into the message queue where it will wait until it can be pushed onto the call stack. When the call stack is empty, the message queue will push the callback onto the stack where it will be carried out. Later, the fetch will return which will initiate its own callback and that will pushed onto the queue. So both things, the callback and fetch, cannot happen simultaneously.

However, that doesn't mean that there still won't be a problem. The callback can still happen before the fetch returns with the data which can cause issues. Solution is to force the callback to wait until the data is loaded.



# HTTP Protocol and Random Web Info

## MIME TYPES

A MIME type is label used to indicate the type of data. It is basically the Internet equivalent of file extensions. MIME types are usually specified in the HTTP headers.



## TCP/IP PORTS AND SOCKETS
1. IP ADDRESS identifies the computer
2. Port identifies the point of access to into the computer
3. A socket refers an ip address and the port (ip address + port) = socket.

An analogy would be the ip address is like the street address of an apartment complex and the port is the specific apartment number
Port numbers are 16-bits so they go from 0-65535.

Port numbers are split into ranges:

Port 0 - 1023: Well-known ports. These ports are allocated to server services by the Internet Assigned Numbers Authority (IANA). Web servers
usually use port 80. These ports are reserved for these kind of uses.

Port 1024 - 49151: Registered Port. Can be registered for services by IANA and are treated as semi-reserved. User programs do not use these ports.

Port 49152 - 65535: Used by client side programs. An internet browser like Chrome for instance will allocate itself to one of these ports.
Also known as ephemeral ports.


WELL KNOWN PORTS NUMBERS
* 21  = FTP
* 22 = SSH
* 443 = HTTP Secure (HTTPS)
* 194 = Internet chat relay (IRC)

## URL (UNIFORM RESOURCE LOCATOR) Syntax

The URL has 4 parts to it:

protocol://hostname:port/path-and-filename-and-additional-parameters

1. Protocol (http, telenet, ftp, whatever is used by the client and server)
2. Hostname: The DNS domain name or IP address of the server
3. Port: The server port that is listening for requests from the client. If it this is not specified, it will default
to the appropriate number for the protocol. For HTTP, the default port number is 80.
4. Path to the requested resource on the server.
5. Additional parameters to help in locating the resource.

Example URL: http://www.example.com/search?item=vw+beetle

Here the port is implicitly set to 80 and the path is set with additional parameters to help with the search.


##INTERNET PROTOCOL SUITE

The Internet protocol suite is built of different processes running at different levels. We can loosely imagine this as a 4-layer model.

1. Application layer. Sits at the top. Applications communicate on this layer making use of service provided by lower layers.
HTTP, SSH, FTP etc. operate on this layer. SSL encryption also works on this layer.
2. Transport layer performs host-to-host communication. This is where TCP/IP and UDP operate. This layer exposes services to the upper layers for use.
3. Internet layer. This is the layer that handles how data is exchange across networks. The internet layer basically hides the actual network topology from the upper layers so they don't have to worry about it. This is where IP addresses are defined.
4. Link layer. This is the general layer that handles how to actually connect different devices together. An example is that the link layer handles how the computer communicates to the modem using ethernet. So the link layer defines the protocol and specifications for how the computer will send/receive info via ethernet to the modem.


## HTTP PROTOCOL

HTTP can use any type of connection to communicate. TCP/IP is one example.

The full process of entering a URL into a browser and getting a webpage can be summarized as follows:

1. Enter URL into browser
2. Browser transforms URL into a HTTP request
3. Browser asks operating system to send the HTTP request to the server indicated in the HTTP request. The server is usuallly listed as the `Host`.
4. Operating system sends a DNS query to look up IP address of the server. DNS = Domain Name System. Basically ask the routing/DNS server to translate the domain name into an IP address for us.
5. Open the appropriate connection (TCP/IP, FTP, etc.) with the computer/server at the IP address.
6. Once connection is open, the operating system sends the HTTP request through the connection.
7. The server is currently listening for requests. Once it gets the request, it will take appropriate action and return the necessary data.

One thing to keep in mind is the use of the word server: does it refer to the actual hardware or the software that is running on the hardware?

### HTTP REQUESTS
When a URL is entered into the browser, it generates a request message.
For example, if we enter the URL: http://www.nowhere123.com/doc/index.html then we generate the following request:

` GET /docs/index.html HTTP/1.1
  Host: www.nowhere123.com
  Accept: image/gif, image/jpeg,
  Accept-Language: en-us
  Accept-Encoding: gzip, deflate
  User-Agent: Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)
  <blank-line>
`

The format of the request message is as follows:

1. Request-line
`GET /docs/index.html HTTP/1.1`

Request-line format: request-method request-URI HTTP-version

request-method: HTTP method for the type of request.
request-URI: filename/specification for the resource requested
HTTP-version: Currently 4 versions

Common request-methods are:
* GET = request a specified resource from the server but the data will be changed in any way
* POST = send data to server to create a resource
* PUT = update existing resource on server using data in the body of the request
* HEAD = similar to GET but the body of the response will be blank. Can be used to check if resource exists before using GET
* TRACE = diagnostic. Response body will contain the exact same content as the request message
* OPTIONS = describe communication options that are available for the target resource
* DELETE = delete resource
* PATCH = apply partial modifications to resource.


2. Request-headers (there can be multiple request headers, each one has its own line)
` Host: www.nowhere123.com
  Accept: image/gif, image/jpeg,
  Accept-Language: en-us
  Accept-Encoding: gzip, deflate
  User-Agent: Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)
`
Request-header format: request-header-name: request-header-value1, request-header-value2, ...

Examples of request-header-names are:
* Host
* Connection
* Accept
* Accept-Language
* Accept-Encoding
* User-Agent

3. Blank-line. This is needed to separate the message body from the headers.

4. Request message body: The request can contain a message body which is separated from the headers by a blank line. The above example doesn't have anything in the request body which is fine.

### HTTP RESPONSES
Now the server can respond in one of 3 ways:

1. Return the requested file to the client
2. Execute a program on the server and return the program output to the client
3. Request denied and error message is returned

The server will then return a response message which can look like:

```
  HTTP/1.1 200 OK
  Date: Sun, 18 Oct 2009 08:56:53 GMT
  Server: Apache/2.2.14 (Win32)
  Last-Modified: Sat, 20 Nov 2004 07:16:26 GMT
  ETag: "10000000565a5-2c-3e94b66c2e680"
  Accept-Ranges: bytes
  Content-Length: 44
  Connection: close
  Content-Type: text/html
  X-Pad: avoid browser bug

  <html><body><h1>It works!</h1></body></html>
```

The format of this response message is as follows:

1. Status line: `HTTP/1.1 200 OK`
Status line format: http-version status-code reason-phrase
The above example has version 1.1, status-code = 200, reason: ok, so request has been processed.

2. Response headers: `Server: Apache/2.2.14 (Win32)`
Response header format: response-header-name: response-value1, response-value2, ...

3. Blank-line: Like the request message, a blank line is used to separate the body from the headers.

4. Response body: This contains the resource that was requested.

### HTML FORM ENCODING

When sending an HTML form, it can have an encoding type (`enctype`):

* `application/x-www-form-urlencoded`: URL-encoded form, the default value if `enctype` is left blank.
* `multipart/form-data`: A multi-part form. This the type of form users use to upload files.
* `text/plain`: As the name suggests, no encoding.

For url-encoded forms, it means that the body of the HTTP request is formatted a certain way. Specifically, it means that the form data is put into the body of the request as a long string of (key, value) pairs. The structure looks like `key1=value1&key2=value2&key3=value3...`. The & sign separates the pairs. If the key or the value includes spaces, then the spaces are changed to '+' or '%20'. So if the key is 'user name', then the encoded version is 'user%20name'.

# NPM AND NODE JS PACKAGES INFO

## NODE JS PACKAGES AND INSTALLING/MANAGING THEM

NPM defines a package as any of the following:

1. a folder containing a program described by a package.json file
2. a gzipped tarball containing (a)
3. a url that resolves to (b)
4. a <name>@<version> that is published on the registry (see npm-registry) with (c)
5. a <name>@<tag> (see npm-dist-tag) that points to (d)
6. a <name> that has a “latest” tag satisfying (e)
7. a <git remote url> that resolves to (a)

Packages should be installed locally since each project may use a different version of the package and global installs can break projects because npm will update the package to the newest version which some projects may not use. Only install packages globally if it provides some sort of global cmd line function or is something that is used across many projects.

Every project that uses node.js should have a package.json file. The package.json file details the dependencies of the project requires. We can install another project's dependences by using npm to install the package.json using the command such as: `npm install <folder containing package.json>`. An example package.json file will have the following info:

```js
{
  "name": "metaverse",
  "version": "0.92.12",
  "description": "The Metaverse virtual reality. The final outcome of all virtual worlds, augmented reality, and the Internet.",
  "main": "index.js"
  "license": "MIT",
  "devDependencies": {
    "mocha": "~3.1",
    "native-hello-world": "^1.0.0",
    "should": "~3.3",
    "sinon": "~1.9"
  },
  "dependencies": {
    "fill-keys": "^1.0.2",
    "module-not-found-error": "^1.0.0",
    "resolve": "~1.1.7"
  }
}

```

Let's discussion semantic versioning. Versions are described by three numbers: #.#.# Above we have 0.92.12. First number is the major version, 2nd number is the minor version, 3rd number is the patch number. Changes in patches usually don't break existing code. Changes to minor version adds new functionality but usually doesn't break anything. Major changes breaks compatibility. There are also special characters when indicating versions.

1. ^ - The carat will install any new minor or patch versions without incrementing the major version. So ^2.4.0 may install 2.4.7 or 2.9.8 but not 3.0.0.
2. ~ - The tilde will install any new patch versions without incrementing the minor version. So if we have ~1.2.7, then any installs of that package can be version 1.2.8 or 1.2.9 but not 1.3.0 since that increments the minor version.
3. latest - if the word 'latest', then npm will just install the newest version
4. * - the asterisk will install any version.

Refer to npm documentation for more information on the special characters. To update the packages based on the semantic versioning, first use `npm outdated` to see which packages have a newer version. Then you can use `npm update <pkg-name>` to update the package. However, `npm update` will not update the package if the major version changes since this implies a break in compatibility. To update to a major version, simply reinstall the package with `npm install <pkg-name>@latest`.

The dependencies are listed below the metadata. The key will be the name of the package and the value is the version of the package. We can differentiate between the types of dependencies. Some dependencies such as `devDependencies` are packages that are used by the developer to help create the project but not used by the project itself. Below is a list of possible other types of dependencies:

* peerDependencies - indicate that the package is compatible with another package so someone can use both packages in the same project without conflict.
* bundledDependencies - indicate the names of packages that will be bundled when you publish your own package


There is also a file called package-lock.json. This is similar to the package.json file but instead details the exact version of each dependency to install. This way when we install using package-lock.json, we won't end up with some other version that might accidentally break our code because the install will be the same every time. Nowadays, npm will override package-lock if package.json has been updated. So if we have both package.json and package-lock.json, then package.json takes precedence. Package-lock.json will automatically be generated whenever npm modifies/updates package.json or the node_modules tree.

To automatically update your package.json whenever you install/remove a dependency use the `--save` option. Therefore, `npm install <pkg> --save` will install the package and update the package.json file in the current directory. `npm uninstall <pkg> --save` will uninstall the package and remove it from the package.json dependency list. Look up npm documentation for more on `--save` options since there is also `--save-dev` for saving something only as a developer dependency.

We can create our own package.json for our project using the command `npm init`. This will bring up several questions about the project which we can answer or skip to be put into the package.json file. To add dependencies to our package.json file, we can run the `npm install <pkg> --save` even if the package is already installed but not included in the package.json file. npm should recognize that package is already installed and will simply update the package.json file.

Packages are NOT just a bunch of src code files. Packages may also be executables or contain executables. If npm installs a package that is an executable, it will put that package in the node_modules/.bin/ folder. A useful tool is `npx`. This is a command-line tool for executing node packages, while npm is for installing/managing node packages. Whenever npm installs an executable, it will create a symlink or a shortcut to the node_modules/.bin/ folder.

npx is used to run CLI tools and other executables hosted on the node.js registry without officially saving them as a package; it will only cache them temporarily. So if you use `npx cowsay` and the package cowsay isn't already installed locally or globally, it will download cowsay, cache it, and run the command. When using `npx` make sure that the current project folder contains `node_modules`.

Whenever you require/install a module, node will look for it by going through all `node_modules` folders in the current and ancestor directories. So it will start with the current directory and look for a `node_modules` folder. If it doesn't find it, it will go up to the parent directory and look there. If it still doesn't find the folder, it will move up the directory chain again. To find out where a `node_modules` folder will be installed, use `npm root`. To find where the nearest executable files are, use `npm bin`.

We can also use npm to run command line scripts. We can specify inside the package.json file a field called `scripts` with a path to the script file to run. Here is an example of that field in a package.json file.

```
{
  "name": "qcapp",
  "version": "0.0.0",
  "private": true,
  "scripts": {
    "start": "node ./bin/www"
  },
  "dependencies": {
    "cookie-parser": "~1.4.4",
    "debug": "~2.6.9",
    "express": "~4.16.1",
    "http-errors": "~1.6.3",
    "morgan": "~1.9.1",
    "pug": "2.0.0-beta11"
  }
}

```

Here we tell npm to run the command line script: `node ./bin/www` which is to just start up node and start running the js file called www. To run a script, use the command `npm run <script-name>`. Here `start` is a special script name that doesn't require us to use the `run` command. We can just use `npm start`.


## JS MODULES STANDARDS

Modular js comes in several different forms. Basically, js, as of this writing, does not inherently support modules and so people attempted to add modularity to js in their own ways. This gave birth to different standards for implementing modules:
  * CommonJS (CJS) = Used in node.js environments. Uses `require` to import modules
  * Asynchronous module definition (AMD)
  * Universal module definition (UMD) - Used in webpack. Can be imported into CJS modules via `require`. Incompatible with ES modules in the sense that you cannot use `import` to import a UMD module.
  * ES Modules (ESM) - ECMAScript modules. Soon to be the default module system in js.

## NODE JS MODULES - BASIC INFO

Every js file in node.js is considered as a module. Within each js file or module, node.js will provide several special objects or functions that are global to that module's scope.

1. `module` - Within the js file, the word module is reserved and is an object that refers to the current js file. For example, if we type the code in the file:

```js

console.log(module.__filename);

```

The console will print out the filename of the current js file. In addition, `module` contains a special property called `module.exports`. This property is an object that will hold the exported code. Basically, when we use the node js `require`, it will go and grab the object that `module.exports` currently references and return it for use. By default, `module.exports` references an empty object literal to which we can add things.

In addition, there is also the keyword `exports`. This is just shorthand that is often used to refer to `module.exports` in order to save on typing. Basically, at the start of each module, node.js will implicitly set `var exports = module.exports`. Now `exports` and `module.exports` both reference/point to the same object. However, this also means that if we later down the line set `exports = {}`, now `exports` points to a new object that is different from the object `module.exports` points to. The key thing to remember is that only the object that `module.exports` references will be returned, not `exports` because `exports` is just a quick hack to save on typing.

We cannot assign to `module.exports` in any callbacks. So we can't use `setTimeOut` to assign the exported code because the modules are loaded immediately. Also, when a module's code is executed, it is actually executed within an anonymous function to encapsulate the scope. This way variables that are global inside the module do not become global inside the other script.

2. `require` - a function unique to node.js that specifies which modules to import. Since it is a function and all functions are objects, it also has properties which we can access. It can also import a JSON data as well.


## NODE JS MODULES - HTTP MODULE

This is the module that enables the http server for node.js. The main application js file must use `require('http')` to use the HTTP server. Recall that node.js by itself is just a js-runtime environment. It can be used for server stuff or not. It is simply a place to run and process js code outside an internet browser.

Let's break down a simple server program:

```js

const http = require('http');
const server = http.createServer();

server.on('request', function(req, res){ //req = object that represents the request, res = object that represents the response
  res.statusCode = 200; //indicate request successful
  res.setHeader('content-type', 'text/plain');
  res.end('Hello World'); //Signal to server that all response headers/body have been sent and server should consider message complete. MUST BE CALLED ON EACH RESPONSE
});

server.on('listening', function(){ //listening event is emitted whenever the server is bound to a port.
  console.log('Server running');
});

server.listen(3000); //start listening for requests on port 3000. 'listening' event is emitted now that server has been bound to port 3000 and console will print 'Server running'
```

Here we **bind** the server to port 3000. Whenever we start a server process, we need to tell the operating system what port number to associate with it. The API that node.js provides for servers is pretty low level. We have to build the request manually (if we are communicating with another server) and we have to write the response manually.


To stop the node.js process from the console, it is better to use 'Ctrl + C'. To stop a process programmatically, we can use `process.exit()`. This will stop the process immediately and any pending/running request is immediately aborted. It is not good practice to stop the process this way. It is better to send a 'SIGTERM' signal. The word signal refers a type of inter-process communication. A signal is a notification sent to a process to notify it that an event has occurred.
    
## EXPRESSJS INFO

ExpressJS is a js framework made to simplify setting up a server in node.js. Node.js provides very low level functionality for server side control which is both good and bad. Good in the sense that almost anything can be done, bad because everything has to be done yourself. The express module is basically a middleware we framework. An express application is essentially a series of middleware function calls to help simplify the development process.

The express application starts by importing the `express` function/object exported by the express module. This function/object has various methods that can be called but also acts as a constructor itself, `express()`. Calling `express()` creates an application object which represents the express application. The `app` object has methods for:

1. Routing HTTP request
2. Configuring middleware
3. Rendering HTML views
4. Registering a template engine

The `app` object is basically a listener for HTTP requests. As such, it can be used when creating `http.createServer(app)`. So we specify the HTTP server should have `app` as the main object to respond to requests.


### EXPRESSJS ROUTING

Routing refers to how an application responds to a certain client request for a particular endpoint, the endpoint being set by the URI and the type of request. In express, each route can have several handler functions which will execute once the server detects the route has been matched; in other words, the URI is correct and the type of request is correct. The express application object handles routing in the following way:

```js

app.METHOD(PATH, HANDLER, next)

```

app = object returned by `express()`
METHOD = HTTP method (post, get, etc.)
path = path to resource ('/' refers to the root path, if path is not specified it default to the root path)
handler = middleware callback function to execute once the route is matched
next = argument representing the next middleware function to be called.

The path can have the asterisk * to indicate a catch-all. So if path = '/*', this means to indicate any path that follows the root path is a valid target to execute the middleware callback.

### EXPRESSJS STATIC FILES

To serve static files, use `app.use(express.static(root, [options]))`. `express.static` is middleware for serving static files. Root = root directory to look for the files, often times being the public folder.  


### EXPRESSJS MIDDLEWARE FUNCTIONS

Middleware is expressjs is just a bunch of functions that have access to the HTTP request (req) and response (res) objects and as well as the `next()` function in the application's request-response cycle. Middleware functions should be able to:

1. Execute any code
2. Make changes to the req and res object
3. End the req-res cycle
4. Call the next middleware function in the stack

Like we said before, expressjs is a framework that is composed of a bunch of middleware functions executed in series, like in a call stack. The middleware can exist in different levels:

* Application level middleware: e.g. `app.use`, stuff we use to carry out our specific application
* Router level: e.g. `router.use`, similar to application level but specific to a specific router
* Error handling
* Built-in
* Third-party middleware

Most of the middleware functions will simply modify the req/res objects by creating a new property to access. We can include middleware functions by using the different methods of the app object as stated above: `app.METHOD(PATH, HANDLER, next)`. Here HANDLER will be our middleware function. We can also use third-party middle-ware with `app.use(PATH, CALLBACK1, CALLBACK2, ...)`. If PATH is not specified, it will default to '/' or the root directory.

To chain middleware functions together, we need to use the argument `next` and call it as a function `next()`. Basically, expressjs will keep track of all the middleware functions we provide to a specific URI. Then, when a request is received for a specific URI, it will execute a middleware function, providing the next middleware function as the `next` argument. This is why we need to call `next()` within a middleware function to move onto the next middleware function because `next` represents the next middleware function. The order of middleware loading matters since whichever middleware function is loaded into expressjs first will be executed first. If we wish to terminate the middleware stack, we simply do not call `next()`.

We can also call `next('route')`. Here 'route' is a literal string, not some placeholder for something else. This will only work in middleware functions loaded by the `app.METHOD()` or `router.METHOD()` functions. Basically, recall that we can load more than one callback to these `app.METHOD()` function. So we can call `app.METHOD(uri, callback1, callback2, ...)`. When you do this, you basically are loading up a sub-stack of middleware functions. Take the following code for instance:

```js

app.get('/user/:id', function start(req, res, next) {
  // if the user ID is 0, skip to the next route
  if (req.params.id === '0') next('route')
  // otherwise pass the control to the next middleware function in this stack
  else next()
}, function middle(req, res, next) {
  // send a regular response
  res.send('regular')
})

// handler for the /user/:id path, which sends a special response
app.get('/user/:id', function end(req, res, next) {
  res.send('special')
})

```

Here the full middle-ware stack for this URI is:

1. a. start
   b. middle
2. end

Here if the id parameter found in start is == 0, then we skip the rest of the sub-stack, meaning we skip the 'middle' function, and move onto the rest of the middleware functions used to handle this URI, which in this case is 'end'.


### EXPRESSJS ROUTER OBJECT

We can call `express.Router()` to create a new router object. This is like a mini-version of our app. The logic is that since expressjs is a bunch of middleware handler functions that respond to different URI paths in our application, the code for our application can get messy very quickly if we have the main application handle everything. Instead, we can structure our application such that our main application delegates the specifics of certain routes to sub-applications which are the Router objects.

## TEMPLATE ENGINE

The point of a template engine is help in the creation of static files. Basically, we can create something like an HTML file and put a bunch of filler variables inside the file. Then the template engine will go ahead and programmatically replace all the filler variables with actual values before sending it off to the client.

## XMLHTTPREQUEST AND FETCH API

XMLHTTPRequest (XHR) is a an api in the form of an object whose methods allow the browser to query data from a server.

The fetch API is a set of tools used to submit HTTP requests and get data back in JSON format. This is similar to XMLHTTPRequest except that fetch uses js Promises to handle callbacks.


## NPM/NODE COMMANDS

`npm start`: Runs the command specified in the package's "start" property of the "scripts" object. If there is no "start" property, it runs `node server.js` instead.

`npm config ls -l`: List all configuration parameters that are internal to npm and are defaults.


## STATE MANAGEMENT

The issue of state management is a complicated one. There are different approaches to how to manage state. Let's discuss the strategy provided by Redux, the state management API associated with React. The basic concept is that the state will be stored in some object, which is typically called `store`. The `store`, or the state, will update itself through a pure function known as a `reducer`, just like the `Array.prototype.reduce` function. The `reducer` simply accepts the `store` object and another object known as an `action` which indicates specifically what action to take to update the state. The `reducer` will then apply the action and return the new updated state. The `store` can only be updated by the developer by calling a `dispatch` method on the `store` object with an `action` argument. The `dispatch` method simply calls the `reducer` with the supplied `action` argument.

The `store` object also provides a publisher-subscribe pattern in which callbacks can be provided to be called whenever a `dispatch` method is called.


## USEFUL PRACTICES SUGGESTIONS

1. Make a script to run/build the app. This allows you to control the version of the commands you want to use to properly build/run the app.

## DIFFERENCE BETWEEN SHELL, CONSOLE, AND TERMINAL

The **terminal** is simply an interface that accepts inputs and passes them along somewhere else and displays any outputs it receives. In the olden days, this might have been a teletypewriter (TTY) which was literally just a typewriter that was also a computer. Nowadays, we use software versions of the traditional terminals. Once again, these software terminals do the exact same thing. They take your input and **pass it on**; they don't do anything else with the inputs.

The **shell** is the program that a terminal sends inputs to. The shell generates outputs and passes it to the terminal which then displays it. The shell is also the program that is used to start other programs. In UNIX, shell refers to a command-line shell which is basically a command-line + shell combined. This means users can submit commands and start other programs using the shell. The shell is the first thing that a user sees when logging in.

The **console** refers to the physical machine that hosts the terminal. But for all practical purposes nowadays, it is basically a terminal since we don't have physical machines that are dedicated to just hosting a terminal. Console emulators basically simulate a terminal and you can change what shell program you wish the terminal to talk to.

Therefore in order to interact with a shell, you need a terminal to send inputs to it. Many shells also double as command line interfaces so you can also execute computer commands using the shell.